In [35]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import torch
import torch.nn.functional as F

In [36]:
image = cv2.imread("samples/C001H001S0001003001.bmp")

In [37]:
tracer_imgs = np.zeros((600, 20, 20, 3))

In [38]:
for t in tqdm(range(600)):
    tracer_imgs[t] = cv2.imread(f"tracer_imgs_light/tracer_{t}.bmp")

100%|██████████| 600/600 [00:00<00:00, 12979.70it/s]


In [ ]:
tmp = tracer_imgs[0]

In [43]:
@torch.no_grad()
def match_template_torch(img, templates):
    res_list = torch.zeros((1, 2))
    img_tensor = torch.from_numpy(img).type(torch.float32)
    for t in tqdm(range(600)):
        template_tensor = torch.from_numpy(templates[t]).type(torch.float32)
        res = match_template_torch_core(img_tensor, template_tensor)
        res_th_96_j, res_th_96_i = torch.where(res > 0.96)

        for n in range(res_th_96_i.size[1]):
            i, j = res_th_96_i[n], res_th_96_j[n]
            for a in range(-5, 6):
                for b in range(-5, 6):
                    if torch.tensor([i+a, j+b]) in res_list:
                        continue

            torch.cat((res_list, torch.tensor([[i, j]])))

    return res_list.cpu().numpy()

@torch.jit.script
def match_template_torch_core(img_tensor, template_tensor):
    result1 = F.conv2d(img_tensor, template_tensor, bias=None, stride=1, padding=0)
    result2 = torch.sqrt(torch.sum(template_tensor**2)* torch.nn.functional.conv2d(img_tensor**2, torch.ones_like(template_tensor), bias=None, stride=1, padding=0))

    return (result1/result2).squeeze(0).squeeze(0)

result = match_template_torch(img=image, templates=tracer_imgs)

  0%|          | 0/600 [00:01<?, ?it/s]


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/var/folders/75/dz10lh0n0yj1snrsvfbf3_9m0000gn/T/ipykernel_53821/1265132780.py", line 23, in match_template_torch_core
@torch.jit.script
def match_template_torch_core(img_tensor, template_tensor):
    result1 = F.conv2d(img_tensor, template_tensor, bias=None, stride=1, padding=0)
              ~~~~~~~~ <--- HERE
    result2 = torch.sqrt(torch.sum(template_tensor**2)* torch.nn.functional.conv2d(img_tensor**2, torch.ones_like(template_tensor), bias=None, stride=1, padding=0))
RuntimeError: expected stride to be a single integer value or a list of 3 values to match the convolution dimensions, but got stride=[1, 1]


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython_darwin_37_64.pyx", line 1035, in _pydevd_bundle.pydevd_cython_darwin_37_64.PyDBFrame.trace_dispatch
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [33]:
result = match_template_torch(img=image, templates=tracer_imgs)

  0%|          | 0/600 [00:00<?, ?it/s]


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/var/folders/75/dz10lh0n0yj1snrsvfbf3_9m0000gn/T/ipykernel_53821/3525179764.py", line 23, in match_template_torch_core
@torch.jit.script
def match_template_torch_core(img_tensor, template_tensor):
    result1 = F.conv2d(img_tensor, template_tensor, bias=None, stride=1, padding=0)
              ~~~~~~~~ <--- HERE
    result2 = torch.sqrt(torch.sum(template_tensor**2)* torch.nn.functional.conv2d(img_tensor**2, torch.ones_like(template_tensor), bias=None, stride=1, padding=0))
RuntimeError: expected stride to be a single integer value or a list of 3 values to match the convolution dimensions, but got stride=[1, 1]
